# Phase 4: Testing model on unseen data 
* The data is manually labelled to test the performance of the model.

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
import sys
import pandas as pd , numpy as np
import re, unicodedata, nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn import metrics
nltk.download('punkt')
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer as tf
import pickle
import joblib
nltk.download('wordnet')
nltk.download('omw-1.4')
from numpy import savetxt
from numpy import asarray
from sklearn.metrics import precision_score, recall_score, accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [35]:
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/MIS798_Files/MIS798_Project/')
from data_pre_processing import remove_numbers, remove_non_ascii, remove_oneandtwo_letter_word, remove_punctuation, remove_stopwords, remove_url, normalize_data

In [36]:
#location = '/content/drive/MyDrive/MIS798_Files/Create_labels/'       # Nishu
location = '/content/drive/MyDrive/Colab Notebooks/MIS798_Files/MIS798_Project/Data_Files/'

### Reading and Processing the test data


In [73]:
test_data = pd.read_csv(location + 'verified_appliances_data.csv')
X_train = pd.read_csv(location+'X_train.csv')  # Reading reviews to fit tf-idf vectorizer
test_data

,Rating,Product_ID,Review_text,Output
0,1,B00006JKZX,unit came on time box looked in great conditio...,Defect
1,4,B00006JKZX,I purchased the Sanyo kegerator as a surprise ...,Defect
2,3,B00006JKZX,I purchased this product based on many hours o...,No defect
3,4,B00006JKZX,This prduct came fairly well shipped. There wa...,No defect
4,4,B00006JKZX,tubing for beer tap is only 3 ft - needs 5 -8 ...,No defect
...,...,...,...,...
295,5,B0002YTLZY,"This duct less vent is great, very quiet, requ...",No defect
296,5,B0002YTLZY,Easy to install great for rental property,No defect
297,5,B0002YTLZY,Awesome,No defect
298,5,B0002YTLZY,Nice replacement oven fan. But the mounting ho...,No defect


In [74]:
test_data['Review_text'] = test_data['Review_text'].apply(lambda x: remove_numbers(x))
test_data['Review_text'] = test_data.apply(lambda row: nltk.word_tokenize(row['Review_text']), axis=1)
test_data['Review_text'] = test_data.apply(lambda row: normalize_data(row['Review_text']), axis=1)
test_data['Review_text'] = test_data.apply(lambda row: remove_oneandtwo_letter_word(row['Review_text']), axis=1)
test_data.loc[test_data['Output'] =='No defect', 'Output'] = 0
test_data.loc[test_data['Output'] == 'Defect','Output'] = 1
test_data

,Rating,Product_ID,Review_text,Output
0,1,B00006JKZX,see metal none since break may worth like ever...,1
1,4,B00006JKZX,birthday immediately shipment bottle every ena...,1
2,3,B00006JKZX,shipment degree heat loading cooler get experi...,0
3,4,B00006JKZX,ship housing perform price small ding draft fa...,0
4,4,B00006JKZX,tube taste tap need well beer,0
...,...,...,...,...
295,5,B0002YTLZY,great duct quiet require wire install less eas...,0
296,5,B0002YTLZY,great install rental property easy,0
297,5,B0002YTLZY,awesome,0
298,5,B0002YTLZY,remove two old maybe mount job person get fan ...,0


In [75]:
X = test_data['Review_text']
y_test = test_data['Output']
X_train = X_train['Text']

### Create vectors for Test data

In [76]:
tfidfvectorizer = tf(analyzer='word',stop_words= 'english')
X_train = tfidfvectorizer.fit_transform(X_train)
x_test = tfidfvectorizer.transform(X)

In [77]:
x_test.shape

(300, 25429)

In [78]:
X_train.shape

(36018, 25429)

### Run saved model on Test data

* <b>Test File</b><br>
Number of No defects = 245<br>
Number of Defects = 55

In [79]:
LR = '/content/drive/MyDrive/Colab Notebooks/MIS798_Files/MIS798_Project/Models/Logistic_Regression_ROS.joblib'

In [80]:
clf_LR = joblib.load(LR)

In [81]:
predictions_LR = clf_LR.predict(x_test)

In [82]:
print(predictions_LR)

[1 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 0 1 1 0 0
 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 1 0 0 1 0
 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 1
 1 0 0 0 1 0 1 1 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1
 1 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0
 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0]


In [85]:
print("No defect predicted = ",len(predictions_LR[predictions_LR==0]))
print("Defect predicted = ",len(predictions_LR[predictions_LR==1]))

No defect predicted =  200
Defect predicted =  100


In [121]:
type(np.int64(y_test[0])), type(predictions_LR[0])

(numpy.int64, numpy.int64)

In [124]:
accuracy = round(accuracy_score(y_test.astype(np.int64), predictions_LR)*100,2)
precision = round(precision_score(y_test.astype(np.int64), predictions_LR)*100,2)
recall = round(recall_score(y_test.astype(np.int64), predictions_LR)*100,2)

print(f'Accuracy = {accuracy} \n Precision = {precision} \n Recall = {recall}')

Accuracy = 75.67 
 Precision = 41.0 
 Recall = 74.55
